In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
%matplotlib inline
%reload_kedro
%load_ext nb_black

2022-04-06 10:01:28,449 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.


/opt/anaconda3/envs/transmilenio/lib/python3.8/site-packages/kedro/framework/context/context.py:32: DeprecationWarning: Accessing package_name via the context will be deprecated in Kedro 0.18.0.
  warn(


2022-04-06 10:01:28,533 - kedro.config.config - INFO - Config from path `/Users/juanazuero/Documents/Transporte/Transmilenio/transmi/conf/local` will override the following existing top-level config keys: dic_estacion_linea_int, dic_estacion_linea_raw, validaciones_troncal, validaciones_troncal_log_int, validaciones_troncal_log_raw, validaciones_troncal_zip_log_int, validaciones_troncal_zip_log_raw
2022-04-06 10:01:28,538 - root - INFO - ** Kedro project transmi
2022-04-06 10:01:28,539 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-04-06 10:01:28,547 - root - INFO - Registered line magic `run_viz`
2022-04-06 10:01:28,600 - blib2to3.pgen2.driver - INFO - Generating grammar tables from /opt/anaconda3/envs/transmilenio/lib/python3.8/site-packages/blib2to3/Grammar.txt
2022-04-06 10:01:28,621 - blib2to3.pgen2.driver - INFO - Writing grammar tables to /Users/juanazuero/Library/Caches/black/20.8b1/Grammar3.8.0.final.0.pickle
2022-04-06 10:01:28,62

<IPython.core.display.Javascript object>

In [3]:
BASE_VALIDACIONES_REQUEST = "https://storage.googleapis.com/validaciones_tmsa/"

<IPython.core.display.Javascript object>

In [7]:
links_data_transmi = catalog.load("links_data_transmi")

2022-04-06 10:48:44,240 - kedro.io.data_catalog - INFO - Loading data from `links_data_transmi` (CSVDataSet)...


<IPython.core.display.Javascript object>

In [5]:
from itertools import product

<IPython.core.display.Javascript object>

In [8]:
result = None
# Iterate over each link or month to extract from the csv in 01_raw
sheet_names_list = [
    "Validaciones Consolidado",
    "VALIDACIONES CONSOLIDADO",
    "VALIDACIONES TULLAVE",
    "Validaciones Tullave",
]
for row in links_data_transmi.iterrows():
    link = row[1]["Link"]
    link_type = row[1]["Type"]
    link = link.replace(" ", "%20")
    if link_type == 1:
        print(link)
        print(str(datetime.datetime.today()))
        xl = pd.ExcelFile(BASE_VALIDACIONES_REQUEST + "ValidacionTroncal/" + link)
        visible_sheets = [
            sh.title for sh in xl.book.worksheets if sh.sheet_state == "visible"
        ]
        sheet = visible_sheets[0]
        for sh in sheet_names_list:
            if sh in visible_sheets:
                sheet = sh
                break
        excel = xl.parse(sheet)
        for i, j in product(range(8), range(8)):
            if str(excel.iloc[i, j]).strip() in ["Fase", "Fase 3"]:
                start_coord = i, j
                break
        cols = excel.iloc[start_coord[0], start_coord[1] :].values
        cols = [
            c.date() if type(c) == datetime.datetime else str(c).strip() for c in cols
        ]
        excel = excel.iloc[start_coord[0] + 1 :, start_coord[1] :].copy()
        excel.columns = cols
        excel.rename({"Fase 3": "Fase"}, axis=1, inplace=True)
        excel = excel[excel["Fase"] != "Total general"].copy()
        excel.drop(["Total general", "Fase"], axis=1, inplace=True)
        excel.dropna(how="all", inplace=True)
        excel.dropna(how="all", axis=1, inplace=True)
        excel_summary = (
            excel.drop(["Línea", "Acceso de Estación"], axis=1)
            .groupby(["Estación", "Intervalo"])
            .sum()
        )
        excel_summary = excel_summary.reset_index().melt(
            id_vars=["Estación", "Intervalo"], var_name="date", value_name="demand"
        )
        result = pd.concat([result, excel_summary], ignore_index=True)

        result["Intervalo"] = result["Intervalo"].apply(
            lambda x: datetime.time(int(x.split(":")[0]), int(x.split(":")[1]))
            if type(x) == str
            else x
        )
        result.to_parquet("prueba.parquet")

2022/02%20TM%20Resumen%20de%20Validaciones%20Troncales%20al%2028%20Feb%202022%20Intervalo%2015%20Mint.xlsx
2022-04-06 10:48:45.495563
2017/TM%20Resumen%20de%20Validaciones%2001%20al%2031%20Diciembre-2017%20cada%2015%20minutos.xlsx
2022-04-06 10:49:21.313959
2017/TM%20Resumen%20de%20Validaciones%2001%20al%2031%20Agosto-2017%20cada%2015%20minutos.xlsx
2022-04-06 10:50:31.987664
2017/TM%20Resumen%20de%20Validaciones%2001%20al%2030%20Septiembre-2017%20cada%2015%20minutos.xlsx
2022-04-06 10:51:38.810358
2017/TM%20Resumen%20de%20Validaciones%2001%20al%2030%20Noviembre-2017%20cada%2015%20minutos.xlsx
2022-04-06 10:52:28.108820


<IPython.core.display.Javascript object>

In [1]:
validaciones_troncal_summary = catalog.load('validaciones_troncal_summary@pandas')

2022-04-08 11:42:22,203 - kedro.io.data_catalog - INFO - Loading data from `validaciones_troncal_summary@pandas` (ParquetDataSet)...


In [2]:
validaciones_troncal_summary

,Estación,Intervalo,date,demand
0,(02000) Cabecera Autopista Norte,00:00:00,2022-02-01,0
1,(02000) Cabecera Autopista Norte,00:15:00,2022-02-01,0
2,(02000) Cabecera Autopista Norte,04:00:00,2022-02-01,1
3,(02000) Cabecera Autopista Norte,04:15:00,2022-02-01,45
4,(02000) Cabecera Autopista Norte,04:30:00,2022-02-01,120
...,...,...,...,...
22444789,(896) L82 Refuerzo CL 79,20:00:00,2017-11-30,1
22444790,(896) L82 Refuerzo CL 79,20:15:00,2017-11-30,0
22444791,(896) L82 Refuerzo CL 79,20:30:00,2017-11-30,0
22444792,(896) L82 Refuerzo CL 79,20:45:00,2017-11-30,0
